In [1]:
import requests
import json

API_KEY = "Q02Na6GOdsZLixhOFcEjeRa1"
SECRET_KEY = "v2w61LHsogLzsJXThxl0A92wpEgz6Nko"


def main(cat, review):

    url = (
        "https://aip.baidubce.com/rpc/2.0/ai_custom/v1/wenxinworkshop/chat/ernie-speed-128k?access_token=https://aip.baidubce.com/rpc/2.0/ai_custom/v1/wenxinworkshop/chat/ernie-speed-128k?access_token="
        + get_access_token()
    )

    payload = json.dumps(
        {
            "messages": [
                {
                    "role": "user",
                    "content": f"以下是在线购物平台对于商品{cat}的评论。请你根据以下评论，判断是正向评价还是负向评价，你只需要回答“正向”或者“负向”，不需要说明理由。{review}",
                },
            ]
        }
    )
    headers = {"Content-Type": "application/json"}

    response = requests.request("POST", url, headers=headers, data=payload)

    return response
    # print(response.text)


def get_access_token():
    """
    使用 AK，SK 生成鉴权签名（Access Token）
    :return: access_token，或是None(如果错误)
    """
    url = "https://aip.baidubce.com/oauth/2.0/token"
    params = {
        "grant_type": "client_credentials",
        "client_id": API_KEY,
        "client_secret": SECRET_KEY,
    }
    return str(requests.post(url, params=params).json().get("access_token"))

In [7]:
import json
import pandas as pd
from tqdm import tqdm


# 检测模型返回的立场
def detect_stance(response_text):
    """
    根据模型返回的文本检测立场
    :param response_text: 模型返回的文本
    :return: 0（负向）或 1（正向）
    """
    if "负向" in response_text:
        return 0
    elif "正向" in response_text:
        return 1
    else:
        return None  # 如果无法判断，返回 None


# 保存运行数据到文件
def save_results(data, filename):
    """
    保存运行数据到 JSON 文件
    :param data: 运行数据的字典列表
    :param filename: 保存文件名
    """
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)


pd_all = pd.read_csv(".\online_shopping_10_cats.csv")

print("评论数目（总体）：%d" % pd_all.shape[0])
print("评论数目（正向）：%d" % pd_all[pd_all.label == 1].shape[0])
print("评论数目（负向）：%d" % pd_all[pd_all.label == 0].shape[0])
pd_positive = pd_all[pd_all.label == 1]
pd_negative = pd_all[pd_all.label == 0]


def get_balance_corpus(corpus_size, corpus_pos, corpus_neg):
    sample_size = corpus_size // 2
    pd_corpus_balance = pd.concat(
        [
            corpus_pos.sample(sample_size, replace=corpus_pos.shape[0] < sample_size),
            corpus_neg.sample(sample_size, replace=corpus_neg.shape[0] < sample_size),
        ]
    )

    print("评论数目（总体）：%d" % pd_corpus_balance.shape[0])
    print(
        "评论数目（正向）：%d"
        % pd_corpus_balance[pd_corpus_balance.label == 1].shape[0]
    )
    print(
        "评论数目（负向）：%d"
        % pd_corpus_balance[pd_corpus_balance.label == 0].shape[0]
    )

    return pd_corpus_balance


# 加载数据集
text_data = get_balance_corpus(1000, pd_positive, pd_negative)


# 初始化统计变量
total_count = 0  # 总数
hit_count = 0  # 命中数
results = []  # 保存每条数据的运行结果

# 遍历数据集
for index, row in tqdm(text_data.iterrows(), total=text_data.shape[0]):
    review = row["review"]  # 评论文本
    cat = row["cat"]  # 分类类别
    true_label = row["label"]  # 真实标签（0 或 1）

    # 调用模型，获取响应
    # 假设 `main(cat, review)` 是你的模型调用函数，这里仅作为占位示例
    response = main(cat, review)  # 替换为你的模型调用代码
    response_data = json.loads(response.text)

    # 提取模型的预测结果
    try:
        extract_response = response_data["result"]  # 模型返回的文本
    except KeyError:
        print("模型返回错误，跳过此条对话：", response_data)
        continue

    # 检测模型预测的立场
    model_stance = detect_stance(extract_response)

    # 如果模型无法检测到立场，跳过此条记录
    if model_stance is None:
        print("无法检测立场，跳过此条记录：", extract_response)
        continue

    # 比较模型预测和真实值
    is_correct = model_stance == true_label
    total_count += 1
    if is_correct:
        hit_count += 1

    # 保存每条记录的运行数据
    results.append(
        {
            "category": cat,
            "review": review,
            "true_label": true_label,
            "model_output": extract_response,
            "detected_stance": model_stance,
            "is_correct": is_correct,
        }
    )

# 计算正确率
accuracy = hit_count / total_count if total_count > 0 else 0

# 保存运行数据到文件
save_results(results, "./results.json")

# 输出统计结果
print(f"总评论数: {total_count}")
print(f"命中数: {hit_count}")
print(f"正确率: {accuracy:.2%}")

<>:32: SyntaxWarning: invalid escape sequence '\o'
<>:32: SyntaxWarning: invalid escape sequence '\o'
C:\Users\11051\AppData\Local\Temp\ipykernel_19472\3811227421.py:32: SyntaxWarning: invalid escape sequence '\o'
  pd_all = pd.read_csv(".\online_shopping_10_cats.csv")


评论数目（总体）：62774
评论数目（正向）：31728
评论数目（负向）：31046
评论数目（总体）：1000
评论数目（正向）：500
评论数目（负向）：500


 60%|██████    | 604/1000 [08:50<05:37,  1.17it/s]

无法检测立场，跳过此条记录： 中性或中立。


 76%|███████▌  | 760/1000 [11:04<03:02,  1.32it/s]

...

无法检测立场，跳过此条记录： 作为一个人工智能语言模型，我还没学习如何回答这个问题，您可以向我问一些其它的问题，我会尽力帮您解决的。


 96%|█████████▌| 960/1000 [13:49<00:31,  1.26it/s]

无法检测立场，跳过此条记录： 中性。


100%|██████████| 1000/1000 [14:22<00:00,  1.16it/s]

总评论数: 996
命中数: 924
正确率: 92.77%
